In [ ]:
import numpy as np
import xarray as xr
import pandas as pd
import geopandas as gpd
from shapely.ops import unary_union
from shapely.geometry import Point

## Apply the mask

### Open the dataset

In [ ]:
root = "/lustre/gmeteo/WORK/DATA/C3S-CDS/C3S-CICA-Atlas/v2/"
domain= "CERRA"
name= "t_CERRA_mon_198501-202112_v02.nc"

In [ ]:
root_nc = root + domain + '/'  + name
var = name.split('_')[0]

ds = xr.open_dataset(root_nc, engine="netcdf4", 
                    chunks={"member": 1, "time": 120})

### Create the mask

In [ ]:
merged_ds = ds.isel(time = 0)

#Create a new variable 'urmask' with NaN values, matching the shape of var
urmask_data = np.nan * np.ones_like(merged_ds[var].values)  # Create NaN-filled array with the same shape as var
merged_ds['urmask'] = (merged_ds[var].dims, urmask_data)
mask = merged_ds.drop_vars(var)

In [ ]:
# Load the shapefile containing the bounding rectangles
shapefile_path = "box/all_cities_bbox.shp"
shapefile_path = "all_cities_bbox.shp"
gdf = gpd.read_file(shapefile_path)

# Function to check if a point is inside any of the bounding rectangles
def is_inside(lat, lon, polygons):
    point = Point(lon, lat)
    return any(poly.contains(point) for poly in polygons)

# Extract geometries from the shapefile (should be a MultiPolygon)
polygons = gdf.geometry

# Iterate over the dataset coordinates
for i in range(mask.lat.shape[0]):
    for j in range(mask.lon.shape[0]):
        if is_inside(mask.lat[i].values, mask.lon[j].values, polygons):
            # Assign value 1 inside the urban area
            mask["urmask"].loc[dict(lat=mask.lat[i].values, lon=mask.lon[j].values)] = 1  

In [ ]:
mask_shape=mask['urmask']

### Apply the mask to the dataset

In [ ]:
ds_masked=ds
ds_masked[var] = ds_masked[var] * mask_shape